In [1]:
pip install zipstream-ai==0.1.3

In [2]:
from zipstream_ai import ZipStreamReader, FileParser, ask

# Step 1: Load ZIP archive
reader = ZipStreamReader("winequality.zip")

# Step 2: List files inside it
print(reader.list_files())

['winequality-white.csv', '__MACOSX/._winequality-white.csv']


In [3]:
# Step 3: Load the CSV inside the ZIP
parser = FileParser(reader)
#df = parser.load("winequality-white.csv")
df = parser.load("winequality-white.csv")
# Step 4: Preview data
print(df.head())

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.0              0.27         0.36            20.7      0.045   
1            6.3              0.30         0.34             1.6      0.049   
2            8.1              0.28         0.40             6.9      0.050   
3            7.2              0.23         0.32             8.5      0.058   
4            7.2              0.23         0.32             8.5      0.058   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 45.0                 170.0   1.0010  3.00       0.45   
1                 14.0                 132.0   0.9940  3.30       0.49   
2                 30.0                  97.0   0.9951  3.26       0.44   
3                 47.0                 186.0   0.9956  3.19       0.40   
4                 47.0                 186.0   0.9956  3.19       0.40   

   alcohol  quality  
0      8.8        6  
1      9.5        6  
2     10.1        6 

In [5]:
import google.generativeai as genai
import pandas as pd

api_key = "<YOUR_GEMINI_API_KEY>"
genai.configure(api_key=api_key)

model = genai.GenerativeModel("models/gemini-1.5-flash-latest")

def ask(df: pd.DataFrame, query: str) -> str:
    prompt = f"""You are given the following pandas dataframe. {query}

{df.head(10).to_markdown()}
"""
    response = model.generate_content(prompt)
    return response.text

In [6]:
question = "Which 3 wine samples have the highest alcohol content?"
response = ask(df, question)
print("\nGemini Response:\n", response)


Gemini Response:
 Here's how to find the 3 wine samples with the highest alcohol content using pandas:

1. **Sort the DataFrame:** We'll sort the DataFrame by the 'alcohol' column in descending order.

2. **Select the top 3:**  We'll then select the first 3 rows of the sorted DataFrame.

Here's the Python code using pandas:

```python
import pandas as pd

data = {'fixed acidity': [7, 6.3, 8.1, 7.2, 7.2, 8.1, 6.2, 7, 6.3, 8.1],
        'volatile acidity': [0.27, 0.3, 0.28, 0.23, 0.23, 0.28, 0.32, 0.27, 0.3, 0.22],
        'citric acid': [0.36, 0.34, 0.4, 0.32, 0.32, 0.4, 0.16, 0.36, 0.34, 0.43],
        'residual sugar': [20.7, 1.6, 6.9, 8.5, 8.5, 6.9, 7, 20.7, 1.6, 1.5],
        'chlorides': [0.045, 0.049, 0.05, 0.058, 0.058, 0.05, 0.045, 0.045, 0.049, 0.044],
        'free sulfur dioxide': [45, 14, 30, 47, 47, 30, 30, 45, 14, 28],
        'total sulfur dioxide': [170, 132, 97, 186, 186, 97, 136, 170, 132, 129],
        'density': [1.001, 0.994, 0.9951, 0.9956, 0.9956, 0.9951, 0.9949,

In [7]:
top_3_alcohol = df.sort_values('alcohol', ascending=False).head(3)
print(top_3_alcohol)

      fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
3918            6.4             0.350         0.28             1.6      0.037   
4503            5.8             0.610         0.01             8.4      0.041   
3904            5.0             0.455         0.18             1.9      0.036   

      free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
3918                 31.0                 113.0  0.98779  3.12       0.40   
4503                 31.0                 104.0  0.99090  3.26       0.72   
3904                 33.0                 106.0  0.98746  3.21       0.83   

      alcohol  quality  
3918    14.20        7  
4503    14.05        7  
3904    14.00        7  
